In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Constants import Const
import Utils
import Models
import Cluster
import re
import Metrics
import json
import glob
import SpatialPreprocessing as spatial
import SymptomPreprocessing as symp
from Levenshtein import distance as levenshtein_distance


In [2]:
md = spatial.MdasiOrganData()
md.process()

invalid patients [551, 552, 558, 559, 566, 762, 769, 813, 507]
spellchecking...
renaming things
filtering pateints
adding histograms
before drop count 823
after drop count 537


/media/andrew/DATA1/git_repos/QubbedDataAnalysis/python/SymptomPreprocessing.py:304: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  val_df[name] = timestep


error (%) 0.7113381521638403
[('severe_6wk_symptoms', 0.00931098696461825), ('severe_late_symptoms', 0.0148975791433892), ('moderate_6wk_symptoms', 0.0521415270018622), ('moderate_late_symptoms', 0.04655493482309125), ('mild_6wk_symptoms', 0.19180633147113593), ('mild_late_symptoms', 0.16759776536312848)]


,concurrent,followup_days,performance_score,baseline_weight,Technique,id,M6_mbs_digest,ic_prior_to_enrollment,chemotherapy,baseline_height,...,GTVn3_centroids,GTVn3_distances,GTVn4_centroids,GTVn4_distances,GTVn5_centroids,GTVn5_distances,GTVn6_centroids,GTVn6_distances,GTVn7_centroids,GTVn7_distances
0,1.0,203.0,0,81.4,IMPT,1,NaN,False,concurrent_chemotherapy,175.0,...,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN
1,1.0,1514.0,0,82.6,IMRT,13,NaN,False,concurrent_chemotherapy,186.0,...,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN
2,1.0,1595.0,0,100.1,IMRT,15,NaN,False,induction+concurrent,178.0,...,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN
3,1.0,1669.0,0,80.1,IMRT,16,NaN,False,concurrent_chemotherapy,178.0,...,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN
4,1.0,1442.0,0,70.1,IMPT,17,NaN,False,concurrent_chemotherapy,157.0,...,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN
5,1.0,1758.0,0,110.4,VMAT,18,NaN,False,concurrent_chemotherapy,166.5,...,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN
6,1.0,1757.0,0,95.5,VMAT,22,3.0,False,induction+concurrent,179.0,...,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN
7,0.0,999.0,0,82.2,IMRT,23,1.0,False,no_chemotherapy,167.0,...,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN
8,1.0,671.0,0,86.2,IMRT,29,1.0,False,concurrent_chemotherapy,180.0,...,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN
9,1.0,775.0,1,83.9,IMRT,30,NaN,False,concurrent_chemotherapy,166.0,...,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN


In [223]:
cd = spatial.CamprtOrganData()
test = cd.process()
test

missing doses set()
missing distances {10153, 205, 10159, 281, 285}
error reading patient 27
'float' object has no attribute 'lower'
renames {'Rt_Sternocleidmastoid_M': 'Rt_Sternocleidomastoid_M'}
renames {'Rt_Sternocleidmastoid_M': 'Rt_Sternocleidomastoid_M'}
renames {'Rt_Sternocleidmastoid_M': 'Rt_Sternocleidomastoid_M'}
renames {'Rt_Sternocleidmastoid_M': 'Rt_Sternocleidomastoid_M'}
renames {'Rt_Sternocleidmastoid_M': 'Rt_Sternocleidomastoid_M'}
renames {'Rt_Sternocleidmastoid_M': 'Rt_Sternocleidomastoid_M'}
error reading patient 10029
bad operand type for unary -: 'str'
invalid patients [17, 39, 101, 115, 239, 10011, 10034, 10043, 10074, 10080, 10094, 10145, 10148, 10164, 10174, 10181, 2020]
0         3
1         4
2        10
3        11
4        29
       ... 
216    2026
217    2027
218    2028
219    5100
220    2030
Name: id, Length: 221, dtype: int64
[3, 4, 10, 11, 29, 31, 33, 34, 35, 36, 37, 41, 46, 49, 100, 102, 104, 105, 109, 110, 112, 114, 118, 119, 120, 121, 125, 126, 12

,id,GTVp_centroids,GTVp_volume,GTVp_min_dose,GTVp_mean_dose,GTVp_max_dose,GTVp_distances,GTVn_centroids,GTVn_volume,GTVn_min_dose,...,bilateral,subsite,rt,ic,cc,nd,locoregional_control,regional_control,local_control,distant_control
0,3,"[37.94251000000003, 15.473832499999958, 2.4037...",1310.0,64.13,71.90,76.19,"[36.53, 26.56, 10.516, 44.9, 20.86, 17.75, 11....","[20.01451000000003, 1.3926324999999338, -7.018...",310.0,68.7,...,False,Tonsil,False,False,True,False,True,True,True,True
1,4,"[-40.05837500000004, 12.823700000000002, 3.622...",1000.0,69.51,70.26,71.46,"[52.62, 26.55, 43.66, 5.457, 32.12, 29.8, 15.2...","[-12.077775000000031, -1.3615999999999815, 1.7...",210.0,68.1,...,False,BOT,False,False,True,False,True,True,True,True
2,10,"[-46.42037250000004, 10.696710000000053, 0.695...",NaN,NaN,72.04,73.45,"[54.53, 36.8, 54.16, 9.734, 38.97, 40.56, 29.1...","[-10.986972500000064, -5.300089999999955, 0.99...",NaN,NaN,...,False,NOS,False,False,True,False,True,True,True,True
3,11,"[21.09186249999999, 4.23923750000003, 2.128789...",3430.0,68.98,71.42,75.23,"[44.44, 23.28, 15.31, 26.52, 26.45, 21.36, 5.8...","[45.348262499999976, 15.801337500000045, 5.280...",520.0,69.63,...,False,BOT,False,False,True,False,True,True,True,True
4,29,"[-40.08905750000005, 19.135957500000046, 2.726...",1410.0,66.14,68.09,70.42,"[49.9, 27.38, 49.78, 9.945, 29.44, 27.62, 10.4...","[-15.23025750000005, -1.1649424999999667, 3.06...",320.0,66.29,...,False,BOT,True,False,False,False,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,2026,"[-21.303897500000033, -0.23330750000002354, -6...",1020.0,69.66,71.36,72.94,"[80.56, 28.5, 63.9, 49.0, 58.8, 50.8, 38.16, 2...","[nan, nan, nan]",NaN,NaN,...,False,Tonsil,False,False,True,False,True,True,True,True
217,2027,"[-39.03744499999999, 11.789114999999981, 6.511...",2200.0,65.6,67.95,69.81,"[44.47, 31.3, 48.06, 9.62, 26.88, 23.7, 16.17,...","[-15.841745000000003, -8.437385000000035, -5.5...",820.0,66.83,...,False,Tonsil,True,False,False,False,True,True,True,True
218,2028,"[-44.14484249999998, 13.539879999999982, 1.525...",3250.0,63.77,72.51,75.16,"[38.72, 29.62, 50.53, 11.72, 19.98, 21.08, 13....","[-64.58674249999999, 56.46567999999996, 5.0962...",20.0,66.79,...,False,Tonsil,False,False,True,True,True,True,True,True
219,5100,"[-7.627272499999975, 4.676557500000058, 5.6432...",2710.0,70.29,71.88,73.93,"[49.25, 20.55, 22.69, 40.06, 30.44, 23.69, 11....","[-40.959472499999976, 15.998357500000054, 2.30...",240.0,68.01,...,False,BOT,False,False,True,False,True,True,True,True


In [21]:
def format_missing_distances(df,key='distances'):
    df = df.copy()
    if 'Extended_Oral_Cavity_'+key not in df.columns:
        print('bad key',key)
        return df
    example = df['Extended_Oral_Cavity_' + key].iloc[0]
    if not Utils.iterable(example):
        print('bad key',key, example)
        return df
    default = np.array([np.NaN for i in enumerate(example)])
    cols = [c for c in df.columns if '_' + key in c]
    for col in cols:
        df[col] = df[col].apply(lambda x: x if (Utils.iterable(x) and len(x) == len(default)) else default)
    return df

format_missing_distances(md.process())

,concurrent,followup_days,performance_score,baseline_weight,Technique,id,M6_mbs_digest,ic_prior_to_enrollment,chemotherapy,baseline_height,...,GTVn3_centroids,GTVn3_distances,GTVn4_centroids,GTVn4_distances,GTVn5_centroids,GTVn5_distances,GTVn6_centroids,GTVn6_distances,GTVn7_centroids,GTVn7_distances
0,1.0,203.0,0,81.4,IMPT,1,NaN,False,concurrent_chemotherapy,175.0,...,"[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,1.0,1514.0,0,82.6,IMRT,13,NaN,False,concurrent_chemotherapy,186.0,...,"[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,1.0,1595.0,0,100.1,IMRT,15,NaN,False,induction+concurrent,178.0,...,"[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
3,1.0,1669.0,0,80.1,IMRT,16,NaN,False,concurrent_chemotherapy,178.0,...,"[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
4,1.0,1442.0,0,70.1,IMPT,17,NaN,False,concurrent_chemotherapy,157.0,...,"[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
5,1.0,1758.0,0,110.4,VMAT,18,NaN,False,concurrent_chemotherapy,166.5,...,"[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
6,1.0,1757.0,0,95.5,VMAT,22,3.0,False,induction+concurrent,179.0,...,"[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
7,0.0,999.0,0,82.2,IMRT,23,1.0,False,no_chemotherapy,167.0,...,"[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
8,1.0,671.0,0,86.2,IMRT,29,1.0,False,concurrent_chemotherapy,180.0,...,"[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, 

In [28]:
def aggregate_gtvs(df,key='distances',aggfunc=None):
    #this probably only works with distance becuase of reasons
    if aggfunc is None:
        if key == 'distances':
            aggfunc = lambda x: np.nanmin(x,axis=0)
        elif key == 'volume':
            aggfunc = lambda x: np.nanmax(x,axis=0)
        else:
            aggfunc = lambda x: np.nanmean(x,axis=0)
    gtv_cols = [c for c in df.columns if 'GTV' in c and '_' + key in c]
    def agg_gtvs(row):
        vals = row[gtv_cols].values
        good_vals = [v for v in vals if not np.all(np.isnan(v))]
        #catch for patients with no gtvs
        if good_vals is None or len(good_vals) < 1:
            return row['GTVp_'+key]
        vals = np.nan_to_num(np.stack(good_vals))
        vals = aggfunc(vals)
        return vals
    return df.apply(agg_gtvs,axis=1)

def add_agg_gtv_vals(df,key='distances'):
    df = df.copy()
    df = format_missing_distances(df,key=key)
    df['GTV_'+key]= aggregate_gtvs(df,key=key)
    return df

def agg_gtvs(df,drop_nodal=True):
    keys = ['distances','centroids']
    df = df.copy()
    for key in keys:
        df = add_agg_gtv_vals(df,key=key)
        if drop_nodal:
            to_drop = [c for c in df.columns if 'GTVn' in c and key in c]
            df = df.drop(to_drop,axis=1)
    return df

agg_gtvs(md.process())

,concurrent,followup_days,performance_score,baseline_weight,Technique,id,M6_mbs_digest,ic_prior_to_enrollment,chemotherapy,baseline_height,...,Rt_Submandibular_Gland_V45,Rt_Submandibular_Gland_V50,Rt_Submandibular_Gland_V55,Rt_Submandibular_Gland_V60,Rt_Submandibular_Gland_V65,Rt_Submandibular_Gland_V70,Rt_Submandibular_Gland_V75,Rt_Submandibular_Gland_V80,GTV_distances,GTV_centroids
0,1.0,203.0,0,81.4,IMPT,1,NaN,False,concurrent_chemotherapy,175.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[49.7, 32.06, 50.72, 19.34, 32.34, 30.6, 20.53...","[-24.63792500000001, 0.1576325000000054, 4.487..."
1,1.0,1514.0,0,82.6,IMRT,13,NaN,False,concurrent_chemotherapy,186.0,...,100.000000,100.000000,99.187500,89.750000,16.859375,0.107849,0.0,0.0,"[51.94, 29.12, 42.06, 18.56, 34.1, 32.7, 19.11...","[-23.904482500000057, 0.7924599999999913, -0.0..."
2,1.0,1595.0,0,100.1,IMRT,15,NaN,False,induction+concurrent,178.0,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,0.0,0.0,"[57.72, 28.75, 28.3, 38.47, 42.53, 37.75, 24.4...","[10.827890000000025, -3.0239375000000166, 1.28..."
3,1.0,1669.0,0,80.1,IMRT,16,NaN,False,concurrent_chemotherapy,178.0,...,100.000000,100.000000,100.000000,100.000000,82.687500,0.000000,0.0,0.0,"[17.61, 24.98, 2.762, 8.2, 7.812, 7.812, 0.976...","[-2.987437499999951, 6.829672499999987, 7.5070..."
4,1.0,1442.0,0,70.1,IMPT,17,NaN,False,concurrent_chemotherapy,157.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[55.53, 27.53, 49.0, 17.33, 35.12, 27.0, 15.73...","[-35.02717000000001, 23.166619999999995, 4.578..."
5,1.0,1758.0,0,110.4,VMAT,18,NaN,False,concurrent_chemotherapy,166.5,...,100.000000,100.000000,100.000000,100.000000,100.000000,99.437500,0.0,0.0,"[51.94, 29.12, 42.06, 18.56, 34.1, 32.7, 19.11...","[-23.904482500000057, 0.7924599999999913, -0.0..."
6,1.0,1757.0,0,95.5,VMAT,22,3.0,False,induction+concurrent,179.0,...,99.750000,98.500000,94.000000,72.875000,19.093750,0.365234,0.0,0.0,"[17.61, 24.98, 2.762, 8.2, 7.812, 7.812, 0.976...","[-2.987437499999951, 6.829672499999987, 7.5070..."
7,0.0,999.0,0,82.2,IMRT,23,1.0,False,no_chemotherapy,167.0,...,100.000000,100.000000,100.000000,94.437500,71.375000,40.593750,0.0,0.0,"[55.53, 27.53, 49.0, 17.33, 35.12, 27.0, 15.73...","[-35.02717000000001, 23.166619999999995, 4.578..."
8,1.0,671.0,0,86.2,IMRT,29,1.0,False,concurrent_chemotherapy,180.0,...,100.000000,100.000000,100.000000,8.585938,0.000000,0.000000,0.0,0.0,"[57.97, 28.08, 15.95, 54.16, 40.7, 38.72, 25.8...","[44.90773249999992, 25.282662499999986, 1.6638..."
9,1.0,775.0,1,83.9,IMRT,30,NaN,False,concurrent_chemotherapy,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[72.1, 24.92, 56.62, 31.44, 52.7, 51.25, 33.1,...","[-27.877584210526322, 3.6154184210526097, -9.9..."


In [224]:
agg_gtvs(cd.process())

,id,GTVp_centroids,GTVp_volume,GTVp_min_dose,GTVp_mean_dose,GTVp_max_dose,GTVp_distances,GTVn_volume,GTVn_min_dose,GTVn_mean_dose,...,rt,ic,cc,nd,locoregional_control,regional_control,local_control,distant_control,GTV_distances,GTV_centroids
0,3,"[37.94251000000003, 15.473832499999958, 2.4037...",1310.0,64.13,71.90,76.19,"[36.53, 26.56, 10.516, 44.9, 20.86, 17.75, 11....",310.0,68.7,71.27,...,False,False,True,False,True,True,True,True,"[36.53, 25.45, 10.516, 44.9, 20.86, 17.75, 11....","[28.97851000000003, 8.433232499999946, -2.3073..."
1,4,"[-40.05837500000004, 12.823700000000002, 3.622...",1000.0,69.51,70.26,71.46,"[52.62, 26.55, 43.66, 5.457, 32.12, 29.8, 15.2...",210.0,68.1,68.78,...,False,False,True,False,True,True,True,True,"[52.62, 26.55, 37.88, 5.457, 32.12, 29.8, 15.2...","[-26.068075000000036, 5.73105000000001, 2.6871..."
2,10,"[-46.42037250000004, 10.696710000000053, 0.695...",NaN,NaN,72.04,73.45,"[54.53, 36.8, 54.16, 9.734, 38.97, 40.56, 29.1...",NaN,NaN,71.97,...,False,False,True,False,True,True,True,True,"[52.66, 23.97, 32.06, 9.734, 33.28, 27.5, 10.0...","[-28.703672500000053, 2.698310000000049, 0.844..."
3,11,"[21.09186249999999, 4.23923750000003, 2.128789...",3430.0,68.98,71.42,75.23,"[44.44, 23.28, 15.31, 26.52, 26.45, 21.36, 5.8...",520.0,69.63,71.95,...,False,False,True,False,True,True,True,True,"[44.44, 23.28, 15.31, 26.52, 26.45, 21.36, 5.8...","[33.22006249999998, 10.020287500000038, 3.7046..."
4,29,"[-40.08905750000005, 19.135957500000046, 2.726...",1410.0,66.14,68.09,70.42,"[49.9, 27.38, 49.78, 9.945, 29.44, 27.62, 10.4...",320.0,66.29,68.14,...,True,False,False,False,True,True,True,True,"[49.9, 27.38, 49.78, 9.945, 29.44, 27.62, 10.4...","[-27.65965750000005, 8.98550750000004, 2.89351..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,2026,"[-21.303897500000033, -0.23330750000002354, -6...",1020.0,69.66,71.36,72.94,"[80.56, 28.5, 63.9, 49.0, 58.8, 50.8, 38.16, 2...",NaN,NaN,NaN,...,False,False,True,False,True,True,True,True,"[80.56, 28.5, 63.9, 49.0, 58.8, 50.8, 38.16, 2...","[-21.303897500000033, -0.23330750000002354, -6..."
217,2027,"[-39.03744499999999, 11.789114999999981, 6.511...",2200.0,65.6,67.95,69.81,"[44.47, 31.3, 48.06, 9.62, 26.88, 23.7, 16.17,...",820.0,66.83,68.49,...,True,False,False,False,True,True,True,True,"[44.47, 23.61, 48.06, 9.62, 26.88, 23.7, 16.17...","[-27.439594999999997, 1.6758649999999733, 0.46..."
218,2028,"[-44.14484249999998, 13.539879999999982, 1.525...",3250.0,63.77,72.51,75.16,"[38.72, 29.62, 50.53, 11.72, 19.98, 21.08, 13....",20.0,66.79,68.04,...,False,False,True,True,True,True,True,True,"[38.72, 29.62, 50.53, 11.72, 19.98, 21.08, 13....","[-54.36579249999998, 35.00277999999997, 3.3110..."
219,5100,"[-7.627272499999975, 4.676557500000058, 5.6432...",2710.0,70.29,71.88,73.93,"[49.25, 20.55, 22.69, 40.06, 30.44, 23.69, 11....",240.0,68.01,70.88,...,False,False,True,False,True,True,True,True,"[27.34, 20.55, 22.69, 15.66, 15.82, 15.625, 11...","[-24.293372499999975, 10.337457500000056, 3.97..."


In [225]:
def spatial_info_df(md,unravel=True,**kwargs):
    pdict = md.process(as_df=True,**kwargs)
    pdict = agg_gtvs(pdict,drop_nodal=True)
    to_keep = ['id']
    organs = ['GTVp','GTV'] + md.organ_list
    for item in ['distances','centroids']:
        to_keep = to_keep + [c + '_' + item for c in organs]
        pdict = format_missing_distances(pdict,key=item)
    df = pdict[to_keep]
    if unravel:
        pairs = set()
        for o1 in organs:
            for idx, name in enumerate(['x','y','z']):
                df[o1+'_'+name] = df[o1+'_centroids'].apply(lambda x: x[idx])
            for idx,o2 in enumerate(md.organ_list):
                if o2+o1 in pairs or o1==o2:
                    continue
                d = df[o1+'_distances'].apply(lambda x: x[idx])
                df[o1+"_"+o2+'_distance'] = d
                pairs.add(o1+o2)
        df = df.drop([c+'_centroids' for c in organs]+[c+'_distances' for c in organs],axis=1)
    return df

#for xinhua
spatial_info_df(md)#.to_csv(Const.data_dir+"MDASI_Distances.csv")

/home/evl/andrew/anaconda3/envs/Qubbed/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/evl/andrew/anaconda3/envs/Qubbed/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/evl/andrew/anaconda3/envs/Qubbed/lib/python3.7/site-packages/ipykernel_launcher.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

,id,GTVp_x,GTVp_y,GTVp_z,GTVp_Esophagus_distance,GTVp_Spinal_Cord_distance,GTVp_Lt_Brachial_Plexus_distance,GTVp_Rt_Brachial_Plexus_distance,GTVp_Cricopharyngeal_Muscle_distance,GTVp_Cricoid_cartilage_distance,...,Lower_Lip_z,Lower_Lip_Upper_Lip_distance,Lower_Lip_Glottic_Area_distance,Upper_Lip_x,Upper_Lip_y,Upper_Lip_z,Upper_Lip_Glottic_Area_distance,Glottic_Area_x,Glottic_Area_y,Glottic_Area_z
0,1,-40.346775,7.228082,4.540130,49.687500,36.000000,54.593750,19.343750,32.343750,30.593750,...,-10.646070,6.000000,87.8125,-2.113675,-67.034517,-27.883070,107.4375,7.707925,1.438682,28.299430
1,13,-34.735883,4.979110,3.287010,51.937500,29.125000,46.562500,18.562500,34.093750,32.687500,...,-10.307390,7.500000,102.8750,2.145017,-63.590190,-24.075490,124.6875,8.339617,8.073410,21.420210
2,15,10.827890,-3.023938,1.280362,57.718750,28.750000,28.296875,38.468750,42.531250,37.750000,...,-2.964337,5.000000,90.5625,1.671490,-72.599038,-15.989138,114.8125,2.079090,5.773062,22.739662
3,16,30.754263,13.329972,-1.283977,17.609375,24.984375,2.761719,8.203125,7.812500,7.812500,...,-3.320778,5.000000,87.8750,1.117463,-85.763628,-17.924278,116.6250,-3.788737,9.975872,21.578222
4,17,-33.034520,29.929170,7.631535,56.593750,31.031250,49.000000,17.328125,37.500000,31.703125,...,-10.929565,6.078125,107.9375,-2.609520,-76.687830,-19.323765,118.7500,-7.894820,10.329170,15.517435
5,18,-34.735883,4.979110,3.287010,51.937500,29.125000,46.562500,18.562500,34.093750,32.687500,...,-10.307390,7.500000,102.8750,2.145017,-63.590190,-24.075490,124.6875,8.339617,8.073410,21.420210
6,22,30.754263,13.329972,-1.283977,17.609375,24.984375,2.761719,8.203125,7.812500,7.812500,...,-3.320778,5.000000,87.8750,1.117463,-85.763628,-17.924278,116.6250,-3.788737,9.975872,21.578222
7,23,-33.034520,29.929170,7.631535,56.593750,31.031250,49.000000,17.328125,37.500000,31.703125,...,-10.929565,6.078125,107.9375,-2.609520,-76.687830,-19.323765,118.7500,-7.894820,10.329170,15.517435
8,29,44.907732,25.282662,1.663843,57.968750,28.078125,15.953125,54.156250,40.687500,38.718750,...,-6.664757,7.500000,88.0625,0.080232,-77.748138,-19.375257,111.6875,-9.815768,2.961162,21.355443
9,30,-36.883784,5.097718,-1.110326,72.125000,37.406250,56.625000,31.437500,52.687500,51.250000,...,-7.086926,7.500000,NaN,1.320816,-72.923582,-21.682126,NaN,NaN,NaN,NaN


In [239]:
def get_val_matrix(df,keys=['distances'],
                   flatten=True,
                   organ_list=None,
                   is_singular = False,
                   fill_na = False,
                  ):
    if organ_list is None:
        organ_list = Const.organ_list[:]
        if 'volume' not in keys and 'mean_dose' not in keys:
            organ_list.append('GTV')
        else:
            organ_list.append('GTVp')
    if 'GTV' in organ_list:
        df = agg_gtvs(df)
        
    if not flatten:
        #check if it's valid not to flattent
        for key in keys:
            array = []
            if key not in ['distances','centroids']:
                organs = [o for o in organ_list if 'GTV' not in o]
                if len(organs) != len(organ_list):
                    print('using a mixture of GTV and non-gtv stuff, so data will be 2-dimensional')
                    flatten=True
                
    col_names = []            
    arraylist=[]
    for key in keys:
        array = []
        make_name =lambda organ: [organ + '_' + key]
        if key not in ['distances','centroids']:
            organs = [o for o in organ_list if 'GTV' not in o]
        else:
            organs= organ_list[:]
            if key == 'distances':
                make_name = lambda organ: [organ + '_' + o2 + '_' + 'distance' for o2 in organs if 'GTV' not in o2]
            else:
                make_name = lambda organ: [organ + '_' + p for p in ['x','y','z']]
        if key in ['volume','mean_dose','max_dose'] + ['V'+str(i) for i in [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,99]]:
            is_singular=True
        if is_singular:
            default = np.nan
        else:
            i = 0
            default = df[organs[0]+'_'+key].iloc[i]
            while not Utils.iterable(default):
                i = i + 1
                default = df[organ_list[0]+'_'+key].iloc[i]
            default = np.array([np.nan for i in default])

        for idx,row in df.iterrows():
            prow = []
            for organ in organs:
                col = organ + "_" + key
                vals = row[col]
                if not is_singular and not Utils.iterable(vals):
                    vals = np.copy(default)
                prow.append(vals)
            array.append(prow)
        for organ in organs:
            names = make_name(organ)
            col_names.extend(names)
        array = np.array(array)
        if fill_na:
            nanidx = np.argwhere(np.isnan(array))
            for idx in nanidx:
                if is_singular:
                    array[idx[0],idx[1]] = np.nanmean(array[:,idx[1]])
                else:
                    array[idx[0],idx[1],idx[2]] = np.nanmean(array[:,idx[1],idx[2]])
        if flatten:
            array = array.reshape((array.shape[0],-1))
        arraylist.append(array)
        
    arraylist = np.concatenate(arraylist,axis=-1)
    if flatten:
        return pd.DataFrame(arraylist,columns = col_names)
    return arraylist

get_val_matrix(md.process(),
              keys=['distances'],
              )

,Esophagus_Esophagus_distance,Esophagus_Spinal_Cord_distance,Esophagus_Lt_Brachial_Plexus_distance,Esophagus_Rt_Brachial_Plexus_distance,Esophagus_Cricopharyngeal_Muscle_distance,Esophagus_Cricoid_cartilage_distance,Esophagus_IPC_distance,Esophagus_MPC_distance,Esophagus_Brainstem_distance,Esophagus_Larynx_distance,...,GTV_Tongue_distance,GTV_Rt_Ant_Digastric_M_distance,GTV_Lt_Ant_Digastric_M_distance,GTV_Mylogeniohyoid_M_distance,GTV_Extended_Oral_Cavity_distance,GTV_Mandible_distance,GTV_Hard_Palate_distance,GTV_Lower_Lip_distance,GTV_Upper_Lip_distance,GTV_Glottic_Area_distance
0,0.0,20.437500,17.656250,20.921875,-1.953125,1.953125,18.437500,38.37500,110.3750,2.794922,...,6.000000,2.429688,14.960938,2.761719,-7.292969,14.351562,57.062500,53.531250,64.25000,32.968750
1,0.0,21.250000,18.484375,20.500000,-2.500000,1.171875,20.843750,38.96875,102.6875,3.427734,...,-6.222656,7.500000,25.265625,12.062500,-9.703125,6.113281,32.156250,56.000000,61.59375,41.750000
2,0.0,24.515625,28.562500,23.546875,-2.343750,1.171875,19.359375,39.50000,122.5625,1.171875,...,-5.000000,15.632812,-1.171875,2.500000,-9.820312,14.968750,50.500000,56.031250,70.68750,37.531250
3,0.0,19.765625,19.406250,22.265625,-1.953125,0.976562,21.468750,42.68750,117.6250,3.087891,...,-13.375000,6.835938,-5.000000,-4.839844,-14.632812,-2.500000,33.281250,34.468750,50.71875,16.906250
4,0.0,19.500000,15.484375,18.250000,3.000000,3.580078,24.953125,49.65625,108.0000,6.308594,...,23.937500,20.968750,47.250000,25.796875,14.828125,7.714844,61.187500,84.250000,86.81250,21.281250
5,0.0,21.250000,18.484375,20.500000,-2.500000,1.171875,20.843750,38.96875,102.6875,3.427734,...,-6.222656,7.500000,25.265625,12.062500,-9.703125,6.113281,32.156250,56.000000,61.59375,41.750000
6,0.0,19.765625,19.406250,22.265625,-1.953125,0.976562,21.468750,42.68750,117.6250,3.087891,...,-13.375000,6.835938,-5.000000,-4.839844,-14.632812,-2.500000,33.281250,34.468750,50.71875,16.906250
7,0.0,19.500000,15.484375,18.250000,3.000000,3.580078,24.953125,49.65625,108.0000,6.308594,...,23.937500,20.968750,47.250000,25.796875,14.828125,7.714844,61.187500,84.250000,86.81250,21.281250
8,0.0,19.109375,18.093750,22.406250,-3.171875,1.953125,19.187500,42.65625,115.9375,1.953125,...,29.703125,53.125000,26.546875,33.750000,22.000000,7.765625,66.500000,80.062500,88.31250,56.281250
9,0.0,20.687500,18.125000,17.984375,-2.929688,-2.183594,16.250000,35.65625,115.0000,-0.976562,...,5.093750,19.453125,49.500000,25.578125,-1.953125,9.984375,19.437500,61.156250,53.87500,0.000000


In [227]:
get_val_matrix(cd.process(),
              keys=['centroids','distances','volume'],
              )

,Esophagus_x,Esophagus_y,Esophagus_z,Spinal_Cord_x,Spinal_Cord_y,Spinal_Cord_z,Lt_Brachial_Plexus_x,Lt_Brachial_Plexus_y,Lt_Brachial_Plexus_z,Rt_Brachial_Plexus_x,...,Tongue_volume,Rt_Ant_Digastric_M_volume,Lt_Ant_Digastric_M_volume,Mylogeniohyoid_M_volume,Extended_Oral_Cavity_volume,Mandible_volume,Hard_Palate_volume,Lower_Lip_volume,Upper_Lip_volume,Glottic_Area_volume
0,2.808510,54.096632,44.041873,-0.537490,53.480832,12.986173,24.256910,46.501932,24.172573,-28.788090,...,2890.0,350.0,370.0,970.0,8900.0,7680.0,320.0,260.0,280.0,10.0
1,7.421425,53.078500,42.724272,0.298325,55.137700,11.900672,29.435125,43.624600,24.446872,-30.782475,...,3510.0,530.0,490.0,1360.0,13700.0,10920.0,500.0,480.0,600.0,50.0
2,5.967127,40.232710,40.750580,-0.697073,50.444210,12.033180,26.961527,32.868510,22.167180,-29.738873,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.071262,39.557238,38.433690,-3.155638,53.392738,10.234990,28.869362,40.434438,20.516490,-36.020238,...,4040.0,380.0,470.0,1440.0,15690.0,8300.0,470.0,360.0,700.0,70.0
4,4.930842,48.664858,44.473910,0.441542,55.991558,15.657810,35.326942,43.993458,28.242010,-28.360258,...,3920.0,610.0,620.0,1840.0,18010.0,10750.0,550.0,540.0,630.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,3.770402,48.066292,46.293000,1.972102,59.411792,21.108600,33.311902,45.357692,28.100400,-29.128098,...,3610.0,500.0,480.0,1480.0,10940.0,9090.0,410.0,460.0,560.0,60.0
217,5.830955,63.156715,42.655190,-0.283145,60.645315,13.169890,28.168255,54.728815,25.891490,-30.914545,...,2900.0,530.0,540.0,1590.0,9980.0,11240.0,430.0,440.0,490.0,70.0
218,3.766657,41.495880,46.668550,-2.629642,56.036980,12.686450,27.083358,44.488680,23.516550,-34.024342,...,3950.0,550.0,580.0,1650.0,10900.0,7870.0,380.0,440.0,660.0,60.0
219,0.945828,56.090558,44.627820,-1.860572,59.621058,13.923420,27.897128,46.942758,22.039020,-32.754672,...,4910.0,560.0,500.0,1670.0,17960.0,11390.0,470.0,550.0,590.0,70.0


In [230]:
from Autoencoders import Normalizer, ConditionalOrganAutoEncoder
from Formatting import *

def merged_spatial_data(cohort1,cohort2,add_id=True,keys=None):
    if keys is None:
        keys = ['centroids','distances','volume']
    c1 = cohort1.process()
    c2 = cohort2.process()
    df = get_val_matrix(c1,
                        keys=keys,
                        flatten=True,
                        fill_na=False)
    df2 = get_val_matrix(cohort2.process(),
                        keys=keys,
                        flatten=True,
                        fill_na=False)
    if add_id:
        df['id'] = c1.id
        df2['id'] = c2.id
    all_cols = [c for c in df.columns if c in df2.columns]
    df = df[all_cols]
    df2 = df2[all_cols]
    df['cohort'] = 0
    df2['cohort'] = 1
    data = pd.concat([df,df2],axis=0,ignore_index=True)
    return data

def get_conditional_autoencoder(x, 
                      train = False,
                      model_path = None, 
                      **kwargs): 
    #takes x np array in form n_items x (dims)
    #assumes missing values are nan and ignores them in training
    #saves model to model_path or default resources dir as autoencoder_<nitems>.pt
    shape = x.reshape((x.shape[0],-1)).shape[-1]
    print(shape)
    autoencoder = ConditionalOrganAutoEncoder( shape )
    
    if model_path is None:
        model_path = pytorch_model_name(x)
        print("autoencoder path not set with original info needed for recreation")
    if not train:
        try: 
            autoencoder.load_state_dict(torch.load(model_path))
        except:
            print("issue loading pretrained autoencoder",model_path,'training...')
            autoencoder = train_autoencoder(autoencoder, x, model_path, **kwargs)
    else:
        autoencoder = train_autoencoder(autoencoder, x, model_path, **kwargs)
        
    autoencoder = autoencoder.eval()
    return autoencoder

def get_imputed_spatial(df,keys=None,**kwargs):
    if 'id' in df.columns: 
        x_original = df.drop(['id'],axis=1).values
    else:
        x_original = df.copy().values
    normalizer = Normalizer(x_original)
    x = normalizer.transform(x_original)
    x = torch.tensor(x).float()
    model_path = pytorch_model_name(x, keys + ['cohort'])
    
    autoencoder = get_conditional_autoencoder(x, model_path = model_path,**kwargs)
    print(autoencoder)
    #check the final loss with dropout
    autoencoder = autoencoder.train(True)
    x_encoded = autoencoder(x)
    print(nan_mse_loss(x_encoded,torch.nan_to_num(x)))
    autoencoder =autoencoder.train(False)
    x_out = x_encoded.cpu().detach().numpy()
    x_out = normalizer.unnormalize(x_out)
    imputed_df = pd.DataFrame(x_out,columns = [c for c in df.columns if c != 'id'])
    imputed_df['id'] = df.id
    return imputed_df

def impute_stuff(md,cd,alpha=0,clip=True,keys=None,**kwargs):
    #alpha is the smoothing term
    if keys is None:
        keys = ['centroids','distances','volume']
    df = merged_spatial_data(md,cd,keys=keys)
    imputed = get_imputed_spatial(df,keys=keys,**kwargs)
    val_cols = [c for c in df.columns if c != 'id' and c != 'cohort']
    
    x = df[val_cols].values
    xi = imputed[val_cols].values
    #denoise with strength alpha.  default alpha = 0 only imputed missing values
    out = (1-alpha)*x + (alpha*xi)
    #fill in nans in orignal data with purely imputed value
    nan_args = np.flatnonzero(np.isnan(x))
    if len(nan_args) > 0:
        shape = out.shape
        out = out.ravel()
        xi = xi.ravel()
        out[nan_args] = xi[nan_args]
        out = out.reshape(shape)
        xi = xi.reshape(shape)
    #clips final values to orignal range
    #the autoencoder shoud do this by default but just to be save
    if clip:
        cleanin = np.nan_to_num(x)
        out - out.clip(min = cleanin.min(axis=0),max = cleanin.max(axis=0))
    out = pd.DataFrame(out,columns = val_cols)
    for col in [c for c in df.columns if c not in out.columns]:
        out[col] = df[col]
    return out

imputed = impute_stuff(md,cd,train=True)
imputed

1804
training stopped on epoch 3595
ConditionalOrganAutoEncoder(
  (hidden_layers): Sequential(
    (0): Linear(in_features=1804, out_features=1500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1500, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=100, bias=True)
    (5): Dropout(p=0.1, inplace=False)
    (6): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Linear(in_features=100, out_features=500, bias=True)
    (9): ReLU()
    (10): Linear(in_features=500, out_features=1500, bias=True)
    (11): ReLU()
    (12): Linear(in_features=1500, out_features=1804, bias=True)
    (13): LeakyReLU(negative_slope=0.1)
  )
  (dropout_layer): Dropout(p=0.6, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)
tensor(0.0202, grad_fn=<MeanBackward0>)


,Esophagus_x,Esophagus_y,Esophagus_z,Spinal_Cord_x,Spinal_Cord_y,Spinal_Cord_z,Lt_Brachial_Plexus_x,Lt_Brachial_Plexus_y,Lt_Brachial_Plexus_z,Rt_Brachial_Plexus_x,...,Lt_Ant_Digastric_M_volume,Mylogeniohyoid_M_volume,Extended_Oral_Cavity_volume,Mandible_volume,Hard_Palate_volume,Lower_Lip_volume,Upper_Lip_volume,Glottic_Area_volume,id,cohort
0,3.574625,39.939483,55.648730,-0.596675,51.666282,18.184930,25.604425,40.757382,33.779030,-28.936575,...,675.404043,1900.228428,20694.391597,6851.895999,365.252922,655.003634,611.289089,10.000003,1,0
1,-0.633483,40.683610,36.588610,-3.706783,47.598710,11.890010,20.142017,39.772110,22.712710,-26.072583,...,5249.977112,18129.348755,183434.486389,92899.799347,4792.213440,3881.454468,5092.620850,793.933868,13,0
2,-3.982110,41.485762,42.599162,-0.295510,50.257562,9.763662,25.770990,40.652262,20.318662,-28.734510,...,4206.390381,14567.871094,103429.412842,89833.831787,3815.002441,4317.626953,7131.500244,791.015625,15,0
3,8.783863,59.583272,43.691823,1.249363,60.951372,12.905523,32.532663,48.563572,21.863022,-28.692437,...,5733.966827,17335.414886,184097.290039,98750.591278,6206.035614,5865.097046,7984.638214,798.702240,16,0
4,5.486680,58.820270,37.889035,1.275380,63.466170,15.422235,27.414380,53.913970,22.222935,-26.397420,...,4488.449951,12440.737949,81348.159159,96007.325106,3484.230759,2847.187187,3017.055166,226.923375,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,3.770402,48.066292,46.293000,1.972102,59.411792,21.108600,33.311902,45.357692,28.100400,-29.128098,...,480.000000,1480.000000,10940.000000,9090.000000,410.000000,460.000000,560.000000,60.000000,2026,1
275,5.830955,63.156715,42.655190,-0.283145,60.645315,13.169890,28.168255,54.728815,25.891490,-30.914545,...,540.000000,1590.000000,9980.000000,11240.000000,430.000000,440.000000,490.000000,70.000000,2027,1
276,3.766657,41.495880,46.668550,-2.629642,56.036980,12.686450,27.083358,44.488680,23.516550,-34.024342,...,580.000000,1650.000000,10900.000000,7870.000000,380.000000,440.000000,660.000000,60.000000,2028,1
277,0.945828,56.090558,44.627820,-1.860572,59.621058,13.923420,27.897128,46.942758,22.039020,-32.754672,...,500.000000,1670.000000,17960.000000,11390.000000,470.000000,550.000000,590.000000,70.000000,5100,1


In [242]:
def roll_up_spatial_df(df,organ_list=None):
    if organ_list is None:
        #assuming this is the inter-organ distance stuff
        organ_list = Const.organ_list[:]
    #also get any GTV columns in there, which don't appear as secondary distances
    base_organs = ['GTVp','GTV','GTVn','GTVn2','GTVn3','GTVn4','GTVn5','GTVn6','GTVn7'] + organ_list
    base_organs = [o for o in base_organs if o+'_x' in df.columns]
    data = {}
    #distances
    for o in base_organs:
        dist_array = []
        for o2 in organ_list:
            dist_col = o + '_' + o2 + '_distance' 
            if dist_col not in df:
                dist_col = o2 + '_' + o + '_distance'
            if dist_col not in df:
                print('missing col',o,o2)
                continue
            dist_vals = df[dist_col].values
            dist_array.append(dist_vals)
        dist_array = np.stack(dist_array,axis=1)
        print(dist_array.shape)
roll_up_spatial_df(imputed)

(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)
(279, 40)


In [236]:
imputed[[c for c in imputed.columns if '_distance' in c]]

,Esophagus_Esophagus_distance,Esophagus_Spinal_Cord_distance,Esophagus_Lt_Brachial_Plexus_distance,Esophagus_Rt_Brachial_Plexus_distance,Esophagus_Cricopharyngeal_Muscle_distance,Esophagus_Cricoid_cartilage_distance,Esophagus_IPC_distance,Esophagus_MPC_distance,Esophagus_Brainstem_distance,Esophagus_Larynx_distance,...,GTVp_Tongue_distance,GTVp_Rt_Ant_Digastric_M_distance,GTVp_Lt_Ant_Digastric_M_distance,GTVp_Mylogeniohyoid_M_distance,GTVp_Extended_Oral_Cavity_distance,GTVp_Mandible_distance,GTVp_Hard_Palate_distance,GTVp_Lower_Lip_distance,GTVp_Upper_Lip_distance,GTVp_Glottic_Area_distance
0,0.0,20.437500,17.656250,20.921875,-1.953125,1.953125,18.437500,38.37500,110.3750,2.794922,...,20.843750,11.757812,45.437500,20.703125,12.070312,14.351562,67.812500,67.06250,74.75000,41.125000
1,0.0,21.250000,18.484375,20.500000,-2.500000,1.171875,20.843750,38.96875,102.6875,3.427734,...,20.531250,8.921875,41.156250,15.007812,11.812500,6.113281,66.187500,74.68750,85.50000,41.750000
2,0.0,24.515625,28.562500,23.546875,-2.343750,1.171875,19.359375,39.50000,122.5625,1.171875,...,-5.000000,15.632812,-1.171875,2.500000,-9.820312,14.968750,50.500000,56.03125,70.68750,37.531250
3,0.0,19.765625,19.406250,22.265625,-1.953125,0.976562,21.468750,42.68750,117.6250,3.087891,...,-13.375000,6.835938,-5.000000,-4.839844,-14.632812,-2.500000,33.281250,43.93750,50.71875,16.906250
4,0.0,19.500000,15.484375,18.250000,3.000000,3.580078,24.953125,49.65625,108.0000,6.308594,...,53.375000,46.625000,66.000000,49.375000,41.531250,41.031250,90.562500,115.12500,118.75000,30.265625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,0.0,21.968750,19.968750,19.359375,-1.953125,0.976562,22.687500,45.37500,104.1875,3.171875,...,-5.484375,9.156250,32.375000,14.781250,-7.343750,11.179688,31.765625,56.40625,60.03125,53.281250
275,0.0,20.218750,16.937500,23.328125,-1.953125,1.953125,21.468750,43.31250,117.5000,4.636719,...,27.828125,13.468750,41.000000,18.000000,18.390625,10.843750,64.875000,75.31250,83.50000,21.750000
276,0.0,22.593750,20.921875,23.421875,2.500000,1.380859,25.000000,47.50000,130.0000,2.683594,...,21.031250,13.828125,46.500000,20.156250,15.492188,0.976562,59.968750,67.68750,77.12500,37.562500
277,0.0,20.218750,19.656250,20.062500,-1.953125,1.953125,21.687500,41.71875,122.5000,3.171875,...,8.242188,0.976562,1.953125,3.521484,-10.781250,18.218750,65.562500,64.87500,80.62500,18.171875


In [234]:
def merged_cohort_stuff(c1,c2):
    df1 = c1.process()
    df2 = c2.process()
    df1['cohort'] = 0
    df2['cohort'] = 1
    cols = [c for c in df1.columns if c in df2.columns]
    return pd.concat([df1[cols],df2[cols]], axis=0,ignore_index=0).reset_index()
merged_cohort_stuff(md,cd)

,index,id,age,t_stage,n_stage,ic,rt,subsite,hpv,GTVp_centroids,...,GTVn3_distances,GTVn4_centroids,GTVn4_distances,GTVn5_centroids,GTVn5_distances,GTVn6_centroids,GTVn6_distances,GTVn7_centroids,GTVn7_distances,cohort
0,0,1,72.000000,t1,n2b,0.0,1.0,BOT,1.0,"[-40.34677500000001, 7.228082499999999, 4.5401...",...,NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,0
1,1,13,73.000000,t2,n2b,0.0,1.0,BOT,1.0,"[-34.73588250000006, 4.9791099999999915, 3.287...",...,NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,0
2,2,15,62.000000,t1,n2b,0.0,1.0,Tonsil,1.0,"[10.827890000000025, -3.0239375000000166, 1.28...",...,NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,0
3,3,16,55.000000,t2,n2c,0.0,1.0,BOT,1.0,"[30.754262500000067, 13.329972499999997, -1.28...",...,NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,0
4,4,17,33.000000,t3,n2c,0.0,1.0,Tonsil,1.0,"[-33.034520000000015, 29.92917, 7.631535000000...",...,NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,216,2026,71.094444,1,0,False,False,Tonsil,True,"[-21.303897500000033, -0.23330750000002354, -6...",...,NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,1
275,217,2027,50.566667,2,0,False,True,Tonsil,True,"[-39.03744499999999, 11.789114999999981, 6.511...",...,NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,1
276,218,2028,55.433333,2,0,False,False,Tonsil,True,"[-44.14484249999998, 13.539879999999982, 1.525...",...,NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,1
277,219,5100,61.275000,2,0,False,False,BOT,True,"[-7.627272499999975, 4.676557500000058, 5.6432...",...,NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,"[nan, nan, nan]",NaN,1


In [116]:
def get_doseval_array(df,
                      dose_keys=None,
                      fill_na=False,
                      dose_organs=None
                     ):
    dose_vals = []
    if dose_keys is None:
        dose_keys= [c for c in MdasiOrganData.default_dose_values if 'volume' not in c]
    for key in dose_keys:
        array = get_val_matrix(df,
                               key=key,
                               is_singular=True,
                               fill_na=fill_na,
                               organ_list=dose_organs,
                               flatten=False,
                                  )
        dose_vals.append(array)
    dose_vals = np.stack(dose_vals,axis=-1)
    return dose_vals

def predict_doses(df, 
                  dose_organs=None,
                  sim_organs=None,
                  sim_matrix= None,
                  sim_model=None,
                  dose_keys=None,
                  knn=None,
                  **kwargs):
    if sim_matrix is None:
        distances = get_val_matrix(df,
                           key='distances',
                           fill_na=True,
                           flatten=False,
                           organ_list=sim_organs,
                          )
        if sim_model is None:
            sim_model = Models.TssimSimilarity()
        sim_matrix = sim_model.get_similarity_matrix(distances)
    
    if knn is None:
        knn = Models.PatientKNN()
    knn_matches = knn.get_matches(sim_matrix)
    
    dval_array = get_doseval_array(df,
                                   dose_organs=dose_organs,
                                   dose_keys=dose_keys)
    predicted_doses = []
    for i,matches in enumerate(knn_matches):
        dose_matches = dval_array[matches]
        avg_doses = np.nanmean(dose_matches,axis=0)
        nanidx = np.argwhere(np.isnan(avg_doses))
        for idx in nanidx:
            avg_doses[idx[0],idx[1]] = np.nanmean(dval_array[:,idx[0],idx[1]])
        predicted_doses.append(avg_doses)
    predicted_doses = np.stack(predicted_doses)
    error = Metrics.masked_mse(predicted_doses,dval_array)
    mae = Metrics.masked_mae(predicted_doses,dval_array)
    entry = {
        'predicted': predicted_doses,
        'mse': error,
        'mae': mae,
        'dose_organs':dose_organs,
        'cluster_organs': sim_organs,
        'features': dose_keys,
        'sim_model': sim_model,
    }
    return entry

sims = [
        Models.TssimSimilarity(),
        Metrics.Jaccard2d(aggregate='average'),
        Metrics.Jaccard2d(aggregate='max'),
        Metrics.Wasserstein2d(aggregate='average'),
        Metrics.Wasserstein2d(aggregate='max'),
        Metrics.Cosine2d(aggregate='average'),
        Metrics.Cosine2d(aggregate='max'),
        Metrics.Euclidean2D(aggregate='average'),
        Metrics.Euclidean2D(aggregate='max'),
       ]
results = []
for sim in sims:
    res = predict_doses(df,
                  sim_model=sim,
                 sim_organs=['GTV']+Const.organ_list,
                dose_keys=['mean_dose'],
                 )
    results.append(res)
pd.DataFrame(results)

NameError: name 'df' is not defined

In [ ]:
def cluster_centroids(df, 
                  sim_organs=None,
                  sim_matrix= None,
                  distances =None,
                  sim_model=None,
                  clusterer=None,
                  n_clusters=4,
                  **kwargs):
    if sim_matrix is None:
        if distances is None:
            distances = get_val_matrix(df,
                               key='distances',
                               fill_na=True,
                               flatten=False,
                               organ_list=sim_organs,
                              )
        if sim_model is None:
            sim_model = Models.TssimSimilarity()
        sim_matrix = sim_model.get_similarity_matrix(distances)
    if clusterer is None:
        clusterer = Cluster.SimilarityClusterer(n_clusters)
    return clusterer.fit_predict(sim_matrix)

cluster_df = df.copy()
for n in [4,5]:
    try:
        cluster_df['jaccard_clusters'+str(n)] = cluster_centroids(df,
                          sim_model=Metrics.Jaccard2d(),
                          sim_organs=["GTV"]+Const.organ_list,
                          n_clusters=n,
                         )
    except Exception as e:
            print(e)
    try:
        cluster_df['tssim_clusters'+str(n)] = cluster_centroids(df,
                          sim_organs=Const.organ_list[:],
                          n_clusters=n,
                         )
    except Exception as e:
            print(e)
    try:
        cluster_df['jaccard_gtv_clusters'+str(n)] = cluster_centroids(df,
                          sim_model=Metrics.Jaccard2d(),
                          sim_organs=["GTV"],
                          n_clusters=n,
                         )
    except Exception as e:
            print(e)
    for sname,slist in Const.symptom_organ_map.items():
        try:
            cluster_df['jaccard_' + sname + '_clusters'+str(n)] = cluster_centroids(df,
                              sim_model=Metrics.Jaccard2d(),
                              sim_organs=slist,
                              n_clusters=n, 
                             )
        except Exception as e:
            print(e)
        try:
            cluster_df['tssim_' + sname + '_clusters'+str(n)] = cluster_centroids(df,
                              sim_organs=slist,
                              n_clusters=n, 
                             )
        except Exception as e:
            print(e)
cluster_df
cluster_df.to_csv('../data/mdasi_distance_clusters.csv',index=False)

In [ ]:
def centroid_df_to_dict(df,
                        oval_keys = None,
                        to_drop = None,
                        organ_list = None):
    if organ_list is None:
        organ_list = Const.organ_list[:]
    if oval_keys is None:
        oval_keys = ['centroids','distances'] 
        oval_keys = oval_keys + MdasiOrganData.default_dose_values[:]
    ocols = [[organ + '_' + key for key in oval_keys] for organ in organ_list]
    if to_drop is None:
        to_drop = ['is_ajcc_8th_edition',
                   'performance_score',
                   't4','n3',
                   'BOT','Tonsil',
                   'old']
    to_drop = set(to_drop).intersection(set(df.columns))
    df = df.copy().drop(to_drop,axis=1)
    isbasecol = lambda x : (not np.any([x in cols for cols in ocols])) and ('GTV' not in x)
    base_cols = [c for c in df.columns if isbasecol(c)]
    gtv_col_bases = [c.replace('_distances','').replace('_centroids','') for c in df.columns if 'GTV' in c]
    ddict = {}
    for pid, row in df.groupby('id'):
        if row.shape[0] > 1:
            print('to big',pid)
        entry = row[base_cols].squeeze(axis=0).to_dict()
        for organ,col_list in zip(organ_list,ocols):
            col_entry = row[col_list].to_dict(orient='records')
            col_entry = [{k.replace(organ+'_',""):v for k,v in e.items()} for e in col_entry]
            entry[organ] = col_entry
        for gtvname in gtv_col_bases:
            gtventry = {}
            flag=True
            for suffix in ('_distances','_centroids'):
                if gtvname+suffix not in row.columns:
                    continue
                val = row[gtvname+suffix].values[0]
                gtventry[suffix.replace('_','')] = val
                if Utils.iterable(val) and np.all([np.isnan(v) for v in val]):
                    flag=False
                elif not Utils.iterable(val) and np.isnan(val):
                    flag=False
            if flag:
                entry[gtvname] = gtventry
        ddict[pid] = entry
    return ddict
 
def save_centroid_dict(df,file=None):
    df = centroid_df_to_dict(df)
    ddict = save_centroid_pdict(df,file=file)
    return ddict

save_centroid_dict(cluster_df)

In [ ]:
[c for c in cluster_df.columns if 'GTV' in c]

In [ ]:
cluster_df.to_dict(orient='records')

In [ ]:
##I think this was old code to get average centroids for other stuff
# avg_centroids = []
# volumes=[]
# organ_dict = {}
# for organ in Const.organ_list:
#     centroids = []
#     vols = []
#     for k,v in pdict.items():
#         entry = v.get(organ)
#         if entry is None:
#             centroid = np.array([np.nan,np.nan,np.nan])
#         else:
#             centroid = entry.get('centroids')
#         centroids.append(centroid)
    
#     centroids = np.stack([np.array(i) for i in centroids])
#     centroid = np.nanmean(centroids,axis=0)
    
#     avg_centroids.append(centroid)
    
# center = np.array(avg_organ_positions).mean(axis=0)
# centroids = np.stack([v - center for v in avg_organ_positions])
# organ_dict = {o:list(v) for o,v in zip(Const.organ_list,centroids)}
# organ_dict
# with open(Const.data_dir + 'organ_centroid.json','w') as f:
#     json.dump(organ_dict,f)
# with open(Const.data_dir + 'organ_centroid.json','r') as f:
#     test = json.load(f)
# test
# fig = plt.figure()
# ax = fig.add_subplot(projection='3d')
# ax.scatter(centroids[:,0],centroids[:,1],centroids[:,2])